In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('/home/sns002/Capstone/data/airbnb-listings_cleaned.csv', low_memory=False)

In [9]:
df.head(5)

,Last Scraped,Experiences Offered,Host Name,Host Since,Host Location,Host Response Time,Host Response Rate,Host Neighbourhood,Host Listings Count,Host Total Listings Count,...,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Cancellation Policy,Calculated host listings count,Reviews per Month,Features
0,2017-04-02,none,Lithal,2014-09-01,"Amsterdam, North Holland, Netherlands",within a day,50.0,Oud-West,1.0,1.0,...,9.0,8.0,9.0,10.0,10.0,9.0,strict,1.0,1.52,"Host Has Profile Pic,Host Identity Verified,Is..."
1,2017-04-02,none,Hein,2014-02-05,"Amsterdam, North Holland, The Netherlands",within a few hours,100.0,Oud-West,1.0,1.0,...,9.0,9.0,9.0,10.0,10.0,9.0,strict,1.0,0.81,"Host Is Superhost,Host Has Profile Pic,Host Id..."
2,2017-04-02,none,Matthieu,2014-11-12,"Amsterdam, North Holland, The Netherlands",Unknown,100.0,Oud-West,1.0,1.0,...,10.0,10.0,10.0,10.0,10.0,10.0,flexible,1.0,0.05,"Host Has Profile Pic,Host Identity Verified,Is..."
3,2017-04-02,none,Bas,2013-10-05,"Amsterdam, North Holland, The Netherlands",within a day,100.0,Oud-West,1.0,1.0,...,10.0,9.0,10.0,9.0,9.0,9.0,flexible,1.0,0.62,"Host Has Profile Pic,Host Identity Verified,Is..."
4,2017-04-02,none,Pablo,2016-08-10,"Amsterdam, North Holland, Netherlands",within a day,100.0,Unknown,1.0,1.0,...,8.0,8.0,8.0,8.0,9.0,9.0,moderate,1.0,2.04,Host Has Profile Pic
